In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont, ImageFilter

In [ ]:
# Load the trained model
model1 = load_model('../models/best_age_model.keras')
model2 = load_model('../models/best_emotion_model.keras')
model3 = load_model('../models/best_gender_model.keras')

In [ ]:
# Load a pre-trained face detector (e.g., Haar cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Gender label dictionary
gender_dict = {0: 'Male \u2642', 1: 'Female \u2640'}

In [ ]:
# Emotion label dictionary with emojis
emotion_labels = {
    'Angry': ('Angry 😠', (255, 0, 0)),  # Red
    'Happy': ('Happy 😃', (0, 255, 0)),  # Green
    'Neutral': ('Neutral 😐', (255, 255, 255)),  # White
    'Sad': ('Sad 😢', (0, 0, 255)),  # Blue
    'Surprise': ('Surprised 😲', (255, 255, 0))  # Yellow
}

In [ ]:
# Function to preprocess the image for prediction
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # Assuming the first detected face is the target
        face = gray[y:y+h, x:x+w]  # Crop the face
    else:
        face = gray  # Use the whole frame if no face is detected
    
    face = cv2.resize(face, (128, 128))  # Resize to model input size
    face = face / 255.0  # Normalize
    face = np.expand_dims(face, axis=-1)  # Add channel dimension
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    
    return face, (x, y, w, h) if len(faces) > 0 else None

In [ ]:
# Function to make predictions for gender, age, and emotion
def predict_gender_age_emotion(img):
    processed_img, face_bbox = preprocess_image(img)

    # Predictions from the three models
    pred_age = model1.predict(processed_img)
    pred_emotion = model2.predict(processed_img)
    pred_gender = model3.predict(processed_img)
    
    # Process gender prediction
    pred_gender_prob = pred_gender[0][0].item()  # Extract scalar using item()
    pred_gender_label = gender_dict[round(pred_gender_prob)]  # Round to nearest gender
    
    # Process age prediction
    pred_age_value = pred_age[0][0].item()  # Extract scalar using item()
    range_width = max(2, int(0.1 * pred_age_value))  # 10% of the predicted age, with a minimum range of 2
    pred_age_lower = max(0, round(pred_age_value - range_width))  # Lower bound
    pred_age_upper = round(pred_age_value + range_width)  # Upper bound
    
    # Process emotion prediction
    emotion_label = list(emotion_labels.keys())[np.argmax(pred_emotion)]
    pred_emotion_label, emotion_color = emotion_labels[emotion_label]  # Get the label with emoji and its color
    
    return pred_gender_label, (pred_age_lower, pred_age_upper), pred_emotion_label, emotion_color, face_bbox

In [ ]:
# Function to apply blur effect on the background behind the text
def apply_blur(frame, background_position):
    # Convert OpenCV image to Pillow Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Extract the region where the blur will be applied
    region = pil_image.crop(background_position)
    
    # Apply Gaussian blur to the region
    blurred_region = region.filter(ImageFilter.GaussianBlur(radius=20))
    
    # Paste the blurred region back onto the image
    pil_image.paste(blurred_region, background_position)
    
    # Convert back to OpenCV
    return cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

In [ ]:
# Path to the Segoe UI Emoji font (on Windows)
font_path = 'C:/Windows/Fonts/seguiemj.ttf'

# Function to draw text with Pillow and apply blur background effect
def draw_text_with_pillow(frame, text, position, font_path=font_path, font_size=30, text_color=(255, 255, 255), padding=10):
    # Convert OpenCV image to Pillow Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)
    
    # Load a font with emoji support
    font = ImageFont.truetype(font_path, font_size)
    
    # Calculate text size and add padding
    text_bbox = draw.textbbox(position, text, font=font)
    text_size = (text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1])
    
    # Define the background position with padding
    background_position = [position[0] - padding, position[1] - padding, position[0] + text_size[0] + padding, position[1] + text_size[1] + padding]
    
    # Apply the blur effect to the region behind the text
    frame = apply_blur(frame, background_position)
    
    # Draw the text with Pillow
    draw.text(position, text, font=font, fill=text_color)  # Colored text
    
    # Convert Pillow Image back to OpenCV
    return cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

In [ ]:
# Open a connection to the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Predict gender, age, and emotion
    pred_gender, (pred_age_lower, pred_age_upper), pred_emotion, emotion_color, face_bbox = predict_gender_age_emotion(frame)
    
    # If a face is detected, draw a bounding box around the face
    if face_bbox is not None:
        (x, y, w, h) = face_bbox
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Draw the predictions using Pillow with better spacing and padding
        frame = draw_text_with_pillow(frame, f'Emotion: {pred_emotion}', (x, y - 120), text_color=emotion_color, font_size=30)
        frame = draw_text_with_pillow(frame, f'Gender: {pred_gender}', (x, y - 80), text_color=(255, 165, 0), font_size=28)
        frame = draw_text_with_pillow(frame, f'Age: {pred_age_lower}-{pred_age_upper}', (x, y - 40), text_color=(255,250,205), font_size=28)
    
    # Display the frame
    cv2.imshow('Gender, Age, and Emotion Prediction', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [28]:
# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()